In [ ]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)


In [10]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://3.238.27.77:7687",
    username="neo4j",
    password="army-acids-fashion"
)

result = graph.query("""
MATCH (m:Movie{title: 'Toy Story'}) 
RETURN m.title, m.plot, m.poster
""")

print(result)

[{'m.title': 'Toy Story', 'm.plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 'm.poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg'}]


In [17]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatOpenAI(openai_api_key= OPENAI_API_KEY, model="gpt-4o-mini")

graph = Neo4jGraph(
    url="bolt://3.238.27.77:7687",
    username="neo4j",
    password="army-acids-fashion"
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

youtube = YouTubeSearchTool()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: e430f805-8f8d-4b26-9966-46c2a419af43


C:\Users\dania\PycharmProjects\neo4j_llm\venv\Lib\site-packages\langsmith\client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
C:\Users\dania\PycharmProjects\neo4j_llm\venv\Lib\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: LAST_MESSAGE)} {position: line: 1, column: 23, offset: 22} for query: 'MATCH (s:`Session`)-[:

Here are two links to the trailers for "The Matrix": [Trailer 1](https://www.youtube.com/watch?v=vKQi3bBA1y8&pp=ygUlRmluZCB0aGUgbW92aWUgdHJhaWxlciBmb3IgdGhlIE1hdHJpeA%3D%3D) and [Trailer 2](https://www.youtube.com/watch?v=9ix7TUGVYIo&pp=ygUlRmluZCB0aGUgbW92aWUgdHJhaWxlciBmb3IgdGhlIE1hdHJpeA%3D%3D). Enjoy!
How can I assist you today?
```
How can I assist you today?
```


Error in RootListenersTracer.on_chain_end callback: BufferError('Existing exports of data: object cannot be re-sized')


How can I assist you today?
```


KeyboardInterrupt: 